In [1]:
%load_ext autoreload
%autoreload 2

from denoised_dataset import HaeaVocab, DenoisingDataset
import torch

vocab = HaeaVocab(torch.zeros(1000, 250, 100), [], max_len=72)

print(len(vocab))

dataset = DenoisingDataset(
    vocab,
    127,
    mask=0.3,
    mask_random=0,
    insert=0,
    rotate=0,
    permute_sentences=0,
    replace_length=-1,
    mask_length='span-poisson',
    poisson_lambda=3
)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe5475cec80>>
Traceback (most recent call last):
  File "/workspace/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


TypeError: HaeaVocab.__init__() missing 2 required positional arguments: 'target_dataset' and 'tgt_var_list'

In [3]:
print(dataset.dataset[0])

tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
        75,  2])


In [7]:
data = dataset[0]
print(data['source'])


tensor([ 1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,  3,  3,  3,  3,
         3,  3, 23, 24, 25, 26, 27,  3,  3,  3,  3, 32, 33, 34, 35, 36, 37,  3,
         3, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59, 60, 61,  3,  3,  3,  3,  3, 67,  3,  3,  3, 71, 72, 73, 74,
        75,  2])


In [32]:
def get_var_seq(var_seq: torch.Tensor, indicate: torch.Tensor):
        # indicate.shape = (batch, max_len + 2)
        result = []
        var_seq = var_seq + len(HaeaVocab.SPECIAL_TOKENS)

        for batch in indicate:
            seq = []
            for item in batch:
                if item in HaeaVocab.SPECIAL_TOKENS:
                    if item == HaeaVocab.SPECIAL_TOKEN_MASK:
                        seq.extend([item for _ in range(len(var_seq))])
                    else:
                        seq.append(item)
                else:
                    seq.extend(var_seq)
                
            result.append(seq)

        result = torch.tensor(result)
        return result


print(get_var_seq(torch.tensor([1, 2, 3]), torch.tensor([[1, 4, 5, 6, 3, 3, 9, 10, 2], [1, 4, 5, 6, 3, 3, 9, 10, 2]])))

tensor([[1, 5, 6, 7, 5, 6, 7, 5, 6, 7, 3, 3, 3, 3, 3, 3, 5, 6, 7, 5, 6, 7, 2],
        [1, 5, 6, 7, 5, 6, 7, 5, 6, 7, 3, 3, 3, 3, 3, 3, 5, 6, 7, 5, 6, 7, 2]])


In [20]:
data = torch.zeros(10, 10)
print(data[0].unsqueeze(0).repeat_interleave(5, dim=0).shape)

torch.Size([5, 10])


In [25]:
def get_tgt_mask(batch, var_len, max_len) -> torch.tensor:
        matrix = torch.zeros(batch, var_len * max_len + 2, var_len * max_len + 2)
        matrix[:, :, 0] = 1

        for i in range(max_len+1):
            s =  (i * var_len) + 1
            e =  ((i+1) * var_len) + 1
            matrix[:, s:e, :e] = 1
        
        matrix[:, -1, :] = 1
        return matrix

In [28]:
print(get_tgt_mask(3, 2, 4))

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1.,